In [50]:
from pymongo import MongoClient
from pymongo.write_concern import WriteConcern
from pymongo import ReadPreference
from pymongo.read_concern import ReadConcern

In [10]:
c = MongoClient('localhost', 27020, directConnection=True)

In [11]:
config = {'_id': 'foo', 'members': [
    {'_id': 0, 'host': 'localhost:27020'},
    {'_id': 1, 'host': 'localhost:27018'},
    {'_id': 2, 'host': 'localhost:27019'}]}

In [12]:
c.admin.command("replSetInitiate", config)

{'ok': 1.0}

2) Продемонструвати запис даних на primary node з різними Write Concern Levels

In [19]:
db = MongoClient("mongodb://localhost:27017,localhost:27018,localhost:27020", replicaSet='foo').test

Unacknowledged

In [34]:
db.products\
    .with_options(write_concern=WriteConcern(w=0))\
    .insert_one({"item": "phone1", "price": 875, "type": "Unacknowledged"},)

Acknowledged

In [35]:
db.products\
    .with_options(write_concern=WriteConcern(w=1))\
    .insert_one({"item": "phone2", "price": 875, "type": "Acknowledged"},)

Journaled

In [36]:
db.products\
    .with_options(write_concern=WriteConcern(w=1, j=True))\
    .insert_one({"item": "phone3", "price": 875, "type": "Journaled"},)

AcknowledgedReplica

In [37]:
db.products\
    .with_options(write_concern=WriteConcern(w="majority"))\
    .insert_one({"item": "phone4", "price": 875, "type": "AcknowledgedReplica"},)

In [38]:
list(db.products.find({}))

[{'_id': ObjectId('6452402803201b95e0a4e2b0'),
  'item': 'phone1',
  'price': 875,
  'type': 'Unacknowledged'},
 {'_id': ObjectId('6452402e03201b95e0a4e2b1'),
  'item': 'phone2',
  'price': 875,
  'type': 'Acknowledged'},
 {'_id': ObjectId('6452402f03201b95e0a4e2b2'),
  'item': 'phone3',
  'price': 875,
  'type': 'Journaled'},
 {'_id': ObjectId('6452403003201b95e0a4e2b3'),
  'item': 'phone4',
  'price': 875,
  'type': 'AcknowledgedReplica'}]

3) Продемонструвати Read Preference Modes: читання з primary і secondary node

In [40]:
# read primary
products = db.get_collection('products', read_preference=ReadPreference.PRIMARY)
products.read_preference

Primary()

In [41]:
list(products.find({}))

[{'_id': ObjectId('6452402803201b95e0a4e2b0'),
  'item': 'phone1',
  'price': 875,
  'type': 'Unacknowledged'},
 {'_id': ObjectId('6452402e03201b95e0a4e2b1'),
  'item': 'phone2',
  'price': 875,
  'type': 'Acknowledged'},
 {'_id': ObjectId('6452402f03201b95e0a4e2b2'),
  'item': 'phone3',
  'price': 875,
  'type': 'Journaled'},
 {'_id': ObjectId('6452403003201b95e0a4e2b3'),
  'item': 'phone4',
  'price': 875,
  'type': 'AcknowledgedReplica'}]

In [42]:
# read secondary
products = db.get_collection('products', read_preference=ReadPreference.SECONDARY)
products.read_preference

Secondary(tag_sets=None, max_staleness=-1, hedge=None)

In [43]:
list(products.find({}))

[{'_id': ObjectId('6452402803201b95e0a4e2b0'),
  'item': 'phone1',
  'price': 875,
  'type': 'Unacknowledged'},
 {'_id': ObjectId('6452402e03201b95e0a4e2b1'),
  'item': 'phone2',
  'price': 875,
  'type': 'Acknowledged'},
 {'_id': ObjectId('6452402f03201b95e0a4e2b2'),
  'item': 'phone3',
  'price': 875,
  'type': 'Journaled'},
 {'_id': ObjectId('6452403003201b95e0a4e2b3'),
  'item': 'phone4',
  'price': 875,
  'type': 'AcknowledgedReplica'}]

4) Спробувати зробити запис з однією відключеною нодою та write concern рівнім 3 та нескінченім таймаутом. Спробувати під час таймаута включити відключену ноду


In [44]:
db.products\
    .with_options(write_concern=WriteConcern(w=3, wtimeout=0))\
    .insert_one({"item": "phone5", "price": 875, "type": "timeout"},)

Поки нода була вимкнута, то запит був у стані виконання. Після ввімкнення ноди знову, через кілька секунд, виконання запиту закінчилося.

In [118]:
list(db.products.find({'item': 'phone5'}))

[{'_id': ObjectId('64524aaf03201b95e0a4e2b4'),
  'item': 'phone5',
  'price': 875,
  'type': 'timeout'}]

5) Аналогічно попередньому пункту, але задати скінченний таймаут та дочекатись його закінчення. Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”


In [47]:
db.products\
    .with_options(write_concern=WriteConcern(w=3, wtimeout=1000))\
    .insert_one({"item": "phone6", "price": 875, "type": "timeout"},)

WTimeoutError: waiting for replication timed out, full error: {'code': 64, 'codeName': 'WriteConcernFailed', 'errmsg': 'waiting for replication timed out', 'errInfo': {'wtimeout': True, 'writeConcern': {'w': 3, 'wtimeout': 1000, 'provenance': 'clientSupplied'}}}

Перевірка чи данні записалися: так

In [48]:
list(db.products.find({'item': 'phone6'}))

[{'_id': ObjectId('64524c4603201b95e0a4e2b5'),
  'item': 'phone6',
  'price': 875,
  'type': 'timeout'}]

Пееревірка чи данні доступні на читання з рівнем readConcern: “majority”. Так, тому що запис не відбувся лише на одну ноду з трьох, а majority не вичитує всі ноди

In [51]:
list(db.products.
     with_options(read_concern=ReadConcern(level="majority"))\
     .find({'item': 'phone6'}))

[{'_id': ObjectId('64524c4603201b95e0a4e2b5'),
  'item': 'phone6',
  'price': 875,
  'type': 'timeout'}]

6) Продемонстрував перевибори primary node в відключивши поточний primary (Replica Set Elections) і що після відновлення роботи старої primary на неї реплікуються нові дані, які з'явилися під час її простою

In [52]:
db.products\
    .with_options(write_concern=WriteConcern(w="majority", wtimeout=0))\
    .insert_one({"item": "phone7", "price": 875, "type": "reelection"},)

Для того, аби перевірити, що дані реплікувалися на старий primary, то було вимкнуто новий primary, та прочитано дані з Read Preference Primary

In [73]:
db.client.address

('localhost', 27020)

In [71]:
list(db.products.
     with_options(read_preference=ReadPreference.PRIMARY)\
     .find({'item': 'phone7'}))

[{'_id': ObjectId('64524f3203201b95e0a4e2b6'),
  'item': 'phone7',
  'price': 875,
  'type': 'reelection'}]

7) Привести кластер до неконсистентного стану користуючись моментом часу коли primary node не відразу помічає відсутність secondary node
 - відключивши дві secondary node протягом 5 сек. на мастер записати значення (з w:1) і перевірити, що воно записалось
 - спробувати зчитати це значення з різними рівнями read concern - readConcern: {level: <"majority"|"local"| "linearizable">}
 - включити дві інші ноди таким чином, щоб вони не бачили попереднього мастера (його можна відключити) і дочекатись поки вони оберуть нового мастера
 - підключити (включити) попередню primary-ноду до кластеру і подивитись, що сталось зі значенням яке було на неї записано


In [75]:
db.products\
    .with_options(write_concern=WriteConcern(w=1, wtimeout=0))\
    .insert_one({"item": "phone9", "price": 1100, "type": "inconsistency"},)

In [80]:
list(db.products.find({'item': 'phone9'}))

[{'_id': ObjectId('64525be503201b95e0a4e2b8'),
  'item': 'phone9',
  'price': 1100,
  'type': 'inconsistency'}]

In [81]:
# read concern: "majority"
list(db.products.
     with_options(read_concern=ReadConcern(level="majority"))\
     .find({'item': 'phone9'}))

[{'_id': ObjectId('64525be503201b95e0a4e2b8'),
  'item': 'phone9',
  'price': 1100,
  'type': 'inconsistency'}]

In [82]:
# read concern: "local"
list(db.products.
     with_options(read_concern=ReadConcern(level="local"))\
     .find({'item': 'phone9'}))

[{'_id': ObjectId('64525be503201b95e0a4e2b8'),
  'item': 'phone9',
  'price': 1100,
  'type': 'inconsistency'}]

In [116]:
# read concern: "linearizable"
list(db.products.
     with_options(read_concern=ReadConcern(level="linearizable"))\
     .find({'item': 'phone9'}))

[{'_id': ObjectId('64525be503201b95e0a4e2b8'),
  'item': 'phone9',
  'price': 1100,
  'type': 'inconsistency'}]

Вимикаємо старий primary та вмикаємо вимкнені ноди. Очікуємо як вони оберуть нового primary. Вмикаємо старого primary та дивимося, що зі записним значенням (воно збереглося)

In [94]:
db.client.address

('localhost', 27020)

In [117]:
list(db.products.
     with_options(read_preference=ReadPreference.PRIMARY)\
     .find({'item': 'phone9'}))

[{'_id': ObjectId('64525be503201b95e0a4e2b8'),
  'item': 'phone9',
  'price': 1100,
  'type': 'inconsistency'}]

8) Земулювати eventual consistency за допомогою установки затримки реплікації для репліки

In [97]:
db.client.address

('localhost', 27020)

In [105]:
c = MongoClient('localhost', 27020, directConnection=True)
config = {
    'version': 123,
    '_id': 'foo', 'members': [
    {'_id': 0, 'host': 'localhost:27020'},
    {'_id': 1,
     'host': 'localhost:27018',
     'priority': 0,
     'hidden': True,
     'secondaryDelaySecs': 120},
    {'_id': 2, 'host': 'localhost:27019'}]}
c.admin.command({'replSetReconfig': config})

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1683120621, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1683120621, 1)}

9) Лишити primary та secondary  для якої налаштована затримка реплікації. Записати декілька значень. Спробувати прочитати значення з readConcern: {level: "linearizable"}
Має бути затримка поки значення не реплікуються на більшість нод


In [113]:
db.products\
    .with_options(write_concern=WriteConcern(w=1, wtimeout=0))\
    .insert_one({"item": "phone11", "price": 500, "type": "delayed"},)

In [122]:
import time

start = time.time()

# read concern: "linearizable"
list(db.products.
     with_options(read_concern=ReadConcern(level="linearizable"))\
     .find({'item': 'phone11'}))

end = time.time()

In [123]:
print(end - start)

120.6792540550232


Як бачимо затримка в часі приблизно така, які в конфігурації